In [1]:
!pip install geopandas
!pip install mgwr
!pip install numba

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 25.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.4 MB/s eta 0:00:00:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 45.8 MB/s eta 0:00:00
  Created wheel for mgwr: filename=mgwr-2.1.2-py3-none-any.whl size=46367 sha256=5125ac2e3fdd5474fd739af121987a9a1e51a0da43f71fc8d3a480240ae90b02
  Stored 

In [2]:
import numpy as np
import pandas as pd
import pickle
import os
import multiprocessing
os.environ['USE_PYGEOS'] = '0'
import geopandas as gp
import multiprocessing as mp
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [17]:
#Load the funda data
funda_data = gp.read_file("data/Houseprices/funda_buy_amsterdam_31-03-2023_full_distances.gpkg")

funda_data = funda_data.loc[funda_data['price'] < 2500000]
# funda_data = pd.DataFrame(funda_data)

# funda_data = gp.GeoDataFrame(pd.get_dummies(funda_data, columns = ["house_type", "building_type", "energy_label", "has_balcony", "has_garden"]))

In [18]:
#show funda data
funda_data


,X,house_id,house_type,building_type,price,price_m2,room,bedroom,bathroom,living_area,...,addresszip,Postcode4,Opp_m2,tram_dist,metro_dist,train_dist,ndvi100,ndvi300,ndvi500,geometry
0,18,42082895,appartement,Resale property,675000,7500.0,3,2,1,90,...,"Tolstraat 60 3, 1073 SC, Amsterdam\n",1073,585996,166.539057,668.095770,1290.089283,0.162001,0.196801,0.195157,POINT (121858.014 484983.965)
1,51,42057103,appartement,Resale property,1295000,6475.0,5,3,0,200,...,"Quellijnstraat 119 C, 1073 XH, Amsterdam\n",1073,585996,208.399329,373.221221,1985.945869,0.142814,0.184581,0.184497,POINT (121381.001 485541.002)
2,79,42038931,appartement,Resale property,385000,7549.0,4,3,0,51,...,"Burgemeester Tellegenstraat 5 II, 1073 KA, Ams...",1073,585996,375.173601,576.663789,1378.133276,0.233371,0.195568,0.208979,POINT (121691.846 484800.990)
3,107,42005049,appartement,Resale property,650000,7738.1,4,3,1,84,...,"Tweede Jan van der Heijdenstraat 38 2, 1073 VJ...",1073,585996,78.048891,689.699113,1524.552850,0.183670,0.184798,0.169328,POINT (121834.000 485376.000)
4,212,42165208,appartement,Resale property,465000,8157.9,3,2,1,57,...,"Mauvestraat 37 H, 1073 RH, Amsterdam\n",1073,585996,152.028798,778.780657,1085.901160,0.215336,0.204926,0.200075,POINT (122003.041 484808.014)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,524,88454958,appartement,Resale property,289000,3524.4,3,2,1,82,...,"Leksmondhof 273, 1108 GA, Amsterdam\n",1108,2919328,3134.295481,304.495796,1894.684593,0.303762,0.319314,0.291643,POINT (127290.990 480243.999)
1715,527,88460312,appartement,Resale property,335000,3764.0,4,3,1,89,...,"Leusdenhof 57, 1108 CV, Amsterdam\n",1108,2919328,3157.747057,465.786814,1740.296080,0.287997,0.326125,0.296571,POINT (127136.997 480186.005)
1716,1009,42065127,appartement,Resale property,299000,4271.4,3,2,1,70,...,"Leusdenhof 44, 1108 CT, Amsterdam\n",1108,2919328,3169.672664,486.295154,1719.114500,0.303994,0.323947,0.300770,POINT (127121.997 480171.002)
1717,1460,42912098,appartement,Resale property,280000,4058.0,3,2,1,69,...,"Leksmondhof 94, 1108 ER, Amsterdam\n",1108,2919328,3074.023871,166.117549,2034.389126,0.367509,0.294422,0.296814,POINT (127397.001 480335.000)


In [19]:
#create array with the dependent variable
b_y = funda_data['price'].values.reshape((-1,1))

In [20]:
# #create an array with the indepentend variables (order matters for the extraction of params later)
# b_X = funda_data[['room', 'bedroom', 'bathroom', 'living_area',
#                   'house_age', 'bus_dist', 'subway_dist', 'train_dist',
#                   'university_dist', 'school_dist', 'mall_dist', 'supermarket_dist',
#                   'energy_label_A', 'energy_label_B', 'energy_label_C', 'energy_label_D',
#                   'energy_label_E', 'energy_label_F', 'energy_label_G', 'energy_label_na',
#                   'has_balcony_0.0', 'has_balcony_1.0', 'has_garden_0.0', 'has_garden_1.0']].values 

cols = ['room', 'bedroom', 'bathroom', 'living_area',
                  'house_age', 'train_dist', 'metro_dist', 'tram_dist', 'ndvi300']

b_X = funda_data[cols].values 

In [21]:
#create coordinate tuple for the model
u = funda_data['geometry'].x
v = funda_data['geometry'].y
b_coords = list(zip(u, v))

In [22]:
multiprocessing.cpu_count()

8

In [23]:
#This might be needed to turn off the OpenMP multi-threading
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


In [24]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).
n_proc = 8 #two processors
pool = mp.Pool(n_proc) 

Process ForkPoolWorker-33:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Process ForkPoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-36:
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-31:
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiproc

In [25]:
%%time
#Run basic GWR in parrallel mode

gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search(pool = pool) #add pool to Sel_BW.search
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit(pool = pool)

120.0
CPU times: user 1.69 s, sys: 168 ms, total: 1.85 s
Wall time: 21.1 s


In [26]:
#show summary
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1657
Number of covariates:                                                    10

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       56255454042155.344
Log-likelihood:                                                  -22440.782
AIC:                                                              44901.564
AICc:                                                             44903.724
BIC:                                                           56255454029946.523
R2:                                                                   0.769
Adj. R2:                                                              0.768

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- -----

In [27]:
%%time
#run MGWR in parrallel mode. Note: max_iter_multi needs to be specified

mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool, max_iter_multi=50, criterion = "AICc", verbose = True) #add pool to Sel_BW.search
print(mgwr_bw)


Backfitting:   0%|          | 0/50 [00:00<?, ?it/s]

Current iteration: 1 ,SOC: 0.0055732
Bandwidths: 43.0, 43.0, 52.0, 71.0, 58.0, 43.0, 43.0, 43.0, 43.0, 43.0
Current iteration: 2 ,SOC: 0.0029242
Bandwidths: 44.0, 44.0, 71.0, 73.0, 68.0, 45.0, 43.0, 44.0, 45.0, 45.0
Current iteration: 3 ,SOC: 0.0020942
Bandwidths: 44.0, 50.0, 75.0, 85.0, 71.0, 62.0, 43.0, 51.0, 50.0, 45.0
Current iteration: 4 ,SOC: 0.0016977
Bandwidths: 50.0, 50.0, 76.0, 88.0, 73.0, 80.0, 45.0, 52.0, 59.0, 49.0
Current iteration: 5 ,SOC: 0.001548
Bandwidths: 52.0, 47.0, 82.0, 94.0, 73.0, 93.0, 49.0, 69.0, 61.0, 59.0
Current iteration: 6 ,SOC: 0.0015395
Bandwidths: 71.0, 45.0, 82.0, 117.0, 73.0, 116.0, 55.0, 81.0, 65.0, 68.0
Current iteration: 7 ,SOC: 0.0014649
Bandwidths: 78.0, 47.0, 110.0, 125.0, 68.0, 116.0, 56.0, 117.0, 66.0, 70.0
Current iteration: 8 ,SOC: 0.0013032
Bandwidths: 86.0, 47.0, 117.0, 139.0, 68.0, 124.0, 56.0, 127.0, 72.0, 76.0
Current iteration: 9 ,SOC: 0.0011724
Bandwidths: 89.0, 47.0, 125.0, 181.0, 68.0, 124.0, 56.0, 139.0, 72.0, 85.0
Current iterati

Inference:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit(pool = pool)


Inference:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [34]:
pickle.dump(mgwr_selector, open('data/models/mgwr_selector.pkl', 'wb'))

In [28]:
#show MGWR model summary
mgwr_results.summary()

NameError: name 'mgwr_results' is not defined

Process ForkPoolWorker-37:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/multiproces

In [48]:
#recreate R format table

def return_geopackage(mgwr_results,cols, path_name):
    df = gp.GeoDataFrame()
    df['Intercept'] = mgwr_results.params[:,0]
    df['intercept_SE'] = mgwr_results.bse[:,0]
    df['intercept_TV'] = mgwr_results.tvalues[:,0]
    
    df['yhat'] = mgwr_results.predy
    df['residual'] = mgwr_results.y.reshape((-1,1)) - mgwr_results.predy
    
    for i,col in enumerate(cols):
        
        df[col] = mgwr_results.params[:,i+1]

        df[col+'_SE'] = mgwr_results.bse[:,i+1]
        df[col+'_TV'] = mgwr_results.tvalues[:,i+1]
        
    df['geometry'] = funda_data['geometry']
    df.to_file(path_name)

In [49]:
#show the GeoDataFrame and write it to a folder
return_geopackage(mgwr_results,cols, 'data/models/mgwr_results_amsterdam_ndvi.gpkg')

In [16]:
#pickle.dump(mgwr_results, open('data/models/gwr_results.pkl', 'wb'))

In [26]:
#gwr_results = pickle.load(open('data/models/mgwr_results.pkl', 'rb'))